In [1]:
# Install dependencies
!pip install pandas numpy matplotlib seaborn scipy wordcloud pyarrow datasets PyGithub python-dotenv --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import re
from collections import Counter
from datetime import datetime
from urllib.parse import urlparse
import time
from github import Github

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Plot settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Environment ready!")

Environment ready!


In [2]:
# Compatibility shim: some versions of fsspec don't expose url_to_fs at top-level.
# This ensures code that expects fsspec.url_to_fs (used by some IO backends) continues to work.
try:
    import fsspec
    if not hasattr(fsspec, "url_to_fs"):
        try:
            from fsspec.core import url_to_fs as _url_to_fs
        except Exception:
            try:
                import fsspec.core as _core
                _url_to_fs = _core.url_to_fs
            except Exception:
                # Fallback shim: create a minimal url_to_fs that returns a filesystem and the path.
                def _url_to_fs(url, **kwargs):
                    protocol = url.split("://")[0] if "://" in url else "file"
                    fs = fsspec.filesystem(protocol)
                    return fs, url
        fsspec.url_to_fs = _url_to_fs
except Exception:
    # If anything goes wrong, continue without failing here; subsequent IO calls will raise their own errors.
    pass

In [3]:
import os
from dotenv import load_dotenv

load_dotenv() 
GITHUB_API_TOKEN = os.environ.get("GITHUB_TOKEN")
gh = Github(GITHUB_API_TOKEN)

/var/folders/9l/dq9r8b6n1bx49mhvn82gzl180000gn/T/ipykernel_23319/4070930473.py:6: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  gh = Github(GITHUB_API_TOKEN)


In [4]:
def extract_owner_repo(repo_url: str, html_url: str):
    for url in (repo_url, html_url):
        if not isinstance(url, str):
            continue
        try:
            path = urlparse(url).path.strip("/")
        except Exception:
            continue

        parts = path.split("/")

        # API: /repos/OWNER/REPO
        if "repos" in parts:
            idx = parts.index("repos")
            if len(parts) >= idx + 3:
                owner = parts[idx + 1]
                repo = parts[idx + 2]
                return owner, repo

        # Web: /OWNER/REPO/pull/123
        if len(parts) >= 2:
            owner = parts[0]
            repo = parts[1]
            return owner, repo

    return None, None

In [5]:
# Load datasets
print("Loading AIDev datasets...")


# Human PRs
human_pr_df = pd.read_parquet("hf://datasets/hao-li/AIDev/human_pull_request.parquet")
human_pr_task_type_df = pd.read_parquet("hf://datasets/hao-li/AIDev/human_pr_task_type.parquet")

perf_hummans = (
    human_pr_df
    .merge(
        human_pr_task_type_df[["id", "type"]], on="id", 
    )
    .query("type == 'perf'")
    .copy()
)

perf_hummans[["repo_owner", "repo_name"]] = perf_hummans.apply(
    lambda row: pd.Series(extract_owner_repo(row['repo_url'], row['html_url'])), axis=1
)

print(f"len = {len(perf_hummans)}")

missing_repos = perf_hummans[perf_hummans['repo_owner'].isna() | perf_hummans['repo_name'].isna()]
print(f"Missing repos: {len(missing_repos)}")

#Reconstruct pr_commit_df from human_pr_df


Loading AIDev datasets...
len = 88
Missing repos: 0


In [6]:
perf_hummans.head()

,id,number,title,user,user_id,state,created_at,closed_at,merged_at,repo_url,html_url,body,agent,type,repo_owner,repo_name
5,2486573779,90516,ref(perf-issues): Consolidate File IO override option,leeandher,35509934,closed,2025-04-28T18:17:36Z,2025-04-28T19:22:01Z,2025-04-28T19:22:01Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/90516,This PR removes the `performance_issues.file_io_main_thread.disabled` override option for the Fi...,Human,perf,getsentry,sentry
12,2419106029,87963,ref(span-buffer): Move from sets to arrays,untitaker,837573,closed,2025-03-26T11:30:27Z,2025-04-04T10:20:54Z,None,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/87963,Arrays might be faster as they might not run comparisons on payloads to\r\ndetermine whether the...,Human,perf,getsentry,sentry
115,2425248848,18585,avoid encoding as double in `napi_create_double` if possible,dylan-conway,35280289,closed,2025-03-28T13:01:23Z,2025-03-28T22:16:32Z,2025-03-28T22:16:32Z,https://api.github.com/repos/oven-sh/bun,https://github.com/oven-sh/bun/pull/18585,### What does this PR do?\r\nArithmetic on numbers encoded as doubles in JSC seems to hit more s...,Human,perf,oven-sh,bun
140,2613893429,20612,Optimize `napi_get_value_string_utf8` `napi_get_value_string_latin1` `napi_get_value_string_ut...,Jarred-Sumner,709451,closed,2025-06-24T07:24:20Z,2025-06-25T00:39:33Z,2025-06-25T00:39:33Z,https://api.github.com/repos/oven-sh/bun,https://github.com/oven-sh/bun/pull/20612,\r\n\r\n### What does this PR do?\r\n\r\nAvoid resolving string slices\r\n\r\nCheck for exceptio...,Human,perf,oven-sh,bun
142,2303501996,16857,Drain microtasks again after deferred tasks run,Jarred-Sumner,709451,closed,2025-01-29T01:31:39Z,2025-03-08T04:09:53Z,None,https://api.github.com/repos/oven-sh/bun,https://github.com/oven-sh/bun/pull/16857,### What does this PR do?\r\n\r\nDeferred tasks might call callbacks to JavaScript. We need to d...,Human,perf,oven-sh,bun


In [ ]:
rows_commits = []
rows_details = []
rows_runs = []
rows_issue_comments = []
rows_review_comments = []
skipped_count = 0

for idx, row in perf_hummans.iterrows():
    pr_id = int(row["id"])
    owner = row["repo_owner"]
    repo_name = row["repo_name"]
    number = int(row["number"])

    full_repo = f"{owner}/{repo_name}"
    print(f"\n➡ Fetching commits, pipelines & comments for {full_repo} PR #{number} (dataset id={pr_id})")

    if pd.isna(owner) or pd.isna(repo_name):
        print("   Skipping due to missing owner/repo")
        skipped_count += 1
        continue

    try:
        repo = gh.get_repo(full_repo)
        pr = repo.get_pull(number)

        pr_title = pr.title
        pr_description = pr.body
        pr_comments_count = pr.comments  

        # ===================== ISSUE COMMENTS  =====================
        try:
            issue_comments = pr.get_issue_comments()
            for c in issue_comments:
                rows_issue_comments.append({
                    "pr_id": pr_id,
                    "pr_number": number,
                    "comment_id": c.id,
                    "user_login": c.user.login if c.user else None,
                    "user_type": c.user.type if c.user else None,
                    "body": c.body,
                    "created_at": c.created_at,
                    "updated_at": c.updated_at,
                    "url": c.html_url,
                })
        except Exception as e_ic:
            print(f"   Error fetching issue comments for PR #{number}: {e_ic}")

        # ===================== REVIEW COMMENTS  =====================
        try:
            review_comments = pr.get_review_comments()
            for rc in review_comments:
                rows_review_comments.append({
                    "pr_id": pr_id,
                    "pr_number": number,
                    "comment_id": rc.id,
                    "user_login": rc.user.login if rc.user else None,
                    "user_type": rc.user.type if rc.user else None,
                    "body": rc.body,
                    "created_at": rc.created_at,
                    "updated_at": rc.updated_at,
                    "path": rc.path,
                    "position": rc.position,
                    "original_position": rc.original_position,
                    "commit_id": rc.commit_id,
                    "original_commit_id": rc.original_commit_id,
                    "in_reply_to_id": getattr(rc, "in_reply_to_id", None),
                    "diff_hunk": rc.diff_hunk,
                    "url": rc.html_url,
                })
        except Exception as e_rc:
            print(f"   Error fetching review comments for PR #{number}: {e_rc}")

        # ===================== COMMITS =====================
        commit_list = pr.get_commits()
        for c in commit_list:
            sha = c.sha
            commit_obj = c.commit

            author_name = None
            committer_name = None
            commit_message = None

            if commit_obj is not None:
                if commit_obj.author is not None:
                    author_name = commit_obj.author.name
                if commit_obj.committer is not None:
                    committer_name = commit_obj.committer.name
                commit_message = commit_obj.message

            stats = c.stats
            commit_stats_additions = getattr(stats, "additions", None)
            commit_stats_deletions = getattr(stats, "deletions", None)
            commit_stats_total = getattr(stats, "total", None)

            # ---- table commits ----
            rows_commits.append({
                "sha": sha,
                "pr_id": pr_id,
                "pr_number": number,
                "repo_owner": owner,
                "repo_name": repo_name,
                "author": author_name,
                "committer": committer_name,
                "commit_message": commit_message,
                "pr_title": pr_title,
                "pr_description": pr_description,
                "pr_comments_count": pr_comments_count,
            })

            # ---- table pr_commit_details ----
            for f in c.files:
                rows_details.append({
                    "sha": sha,
                    "pr_id": pr_id,
                    "pr_number": number,
                    "commit_stats_total": commit_stats_total,
                    "commit_stats_additions": commit_stats_additions,
                    "commit_stats_deletions": commit_stats_deletions,
                    "filename": f.filename,
                    "status": f.status,
                    "additions": f.additions,
                    "deletions": f.deletions,
                    "changes": f.changes,
                    "patch": f.patch,
                })

        # ===================== PIPELINES / WORKFLOW RUNS =====================
        head_sha = pr.head.sha
        head_branch = pr.head.ref

        for run in repo.get_workflow_runs(branch=head_branch, event="pull_request"):
            if run.head_sha != head_sha:
                continue

            rows_runs.append({
                "run_id": run.id,
                "pr_id": pr_id,
                "pr_number": number,
                "workflow_id": run.workflow_id,
                "workflow_name": getattr(run, "name", None),
                "head_branch": run.head_branch,
                "head_sha": run.head_sha,
                "event": run.event,
                "status": run.status,
                "conclusion": run.conclusion,
                "created_at": run.created_at,
                "updated_at": run.updated_at,
                "run_attempt": getattr(run, "run_attempt", None),
                "url": run.html_url,
            })

        time.sleep(0.7)

    except Exception as e:
        print(f"   Error fetching PR #{number} from {full_repo}: {e}")
        skipped_count += 1

# ===================== DATAFRAMES =====================
df_commits = pd.DataFrame(rows_commits)
df_details = pd.DataFrame(rows_details)
df_runs = pd.DataFrame(rows_runs)
df_issue_comments = pd.DataFrame(rows_issue_comments)
df_review_comments = pd.DataFrame(rows_review_comments)

print("\nTotal commit rows (human_pr_commits):", len(df_commits))
print("Total detail rows (human_pr_commit_details):", len(df_details))
print("Total run rows (human_pr_workflow_runs):", len(df_runs))
print("Total issue comment rows (human_pr_issue_comments):", len(df_issue_comments))
print("Total review comment rows (human_pr_review_comments):", len(df_review_comments))

# ===================== FILTERS =====================
print("\nApplying filters to commit data...")

if len(df_details) == 0:
    print("  No commit detail rows; skipping filters.")
    filtered_pr_ids = set(df_commits.get('pr_id', pd.Series(dtype=int)).unique())
else:
    print(f"  Starting detail rows: {len(df_details):,}")


    before_filename = len(df_details)
    df_details = df_details[df_details['filename'].notna()].copy()
    print(f"  Filtered out null filenames: {before_filename - len(df_details):,} records removed")
    print(f"  Remaining after filename filter: {len(df_details):,} detail rows")

    config_patterns = [
        r'^\.mvn/',
        r'^\.gradle/',
        r'^\.idea/',
        r'^\.vscode/',
        r'^\.github/workflows/',
        r'\.properties$',
        r'\.xml$',
        r'\.yml$',
        r'\.yaml$',
        r'\.json$',
        r'\.md$',          
        r'\.txt$',
        r'\.gitignore$',
        r'\.dockerignore$',
        r'/Dockerfile$',
        r'^Dockerfile$',
        r'/docker-compose',
        r'^docker-compose',
        r'\.lock$',
        r'^LICENSE',
        r'^README',
    ]
    config_pattern = '|'.join(config_patterns)

    before_config_filter = len(df_details)

    df_details['is_config_file'] = df_details['filename'].str.contains(
        config_pattern, case=False, na=False, regex=True
    )
    df_details['is_code_file'] = ~df_details['is_config_file']

    pr_has_code = df_details.groupby('pr_id')['is_code_file'].any().reset_index()
    pr_has_code.columns = ['pr_id', 'has_code_files']

    df_details = df_details.merge(pr_has_code, on='pr_id', how='left')
    df_details = df_details[df_details['has_code_files']].copy()
    df_details = df_details.drop(columns=['is_config_file', 'is_code_file', 'has_code_files'])

    print(f"  Filtered out config-only PR/file rows: {before_config_filter - len(df_details):,} records removed")
    print(f"  Remaining after config filter: {len(df_details):,} detail rows")

    if 'commit_message' in df_commits.columns:
        before_merge_filter = len(df_details)

        merge_patterns = [
            r'^Merge\s+branch',
            r'^Merge\s+pull\s+request',
            r'^Merge\s+remote-tracking\s+branch',
            r'^Merge\s+.*\s+into\s+',
            r"^Merged\s+in\s+",
        ]
        merge_pattern = '|'.join(merge_patterns)

        merge_shas = set(
            df_commits[
                df_commits['commit_message'].str.match(merge_pattern, case=False, na=False)
            ]['sha'].tolist()
        )

        if merge_shas:
            df_commits = df_commits[~df_commits['sha'].isin(merge_shas)].copy()
            df_details = df_details[~df_details['sha'].isin(merge_shas)].copy()

        after_merge_filter = len(df_details)
        print(f"  Filtered out merge commits: {before_merge_filter - after_merge_filter:,} detail rows removed from {len(merge_shas)} merge commit(s)")
        print(f"  Remaining after merge filter: {after_merge_filter:,} detail rows")
    else:
        print("  ⚠ 'commit_message' column missing in df_commits; skipping merge filter.")

    # 4) PRs que quedan después de TODOS los filtros de detalles
    filtered_pr_ids = set(df_details['pr_id'].unique())
    print(f"  PRs remaining after all detail-level filters: {len(filtered_pr_ids):,}")

# ===================== SINCRONIZAR df_commits / runs / comments =====================  # NEW
if filtered_pr_ids:
    df_commits = df_commits[df_commits['pr_id'].isin(filtered_pr_ids)].copy()
    df_runs = df_runs[df_runs['pr_id'].isin(filtered_pr_ids)].copy()
    df_issue_comments = df_issue_comments[df_issue_comments['pr_id'].isin(filtered_pr_ids)].copy()
    df_review_comments = df_review_comments[df_review_comments['pr_id'].isin(filtered_pr_ids)].copy()

    print("\nAfter syncing to filtered PRs:")
    print("  Commit rows:", len(df_commits))
    print("  Detail rows:", len(df_details))
    print("  Run rows:", len(df_runs))
    print("  Issue comment rows:", len(df_issue_comments))
    print("  Review comment rows:", len(df_review_comments))
else:
    print("\n⚠ No PRs left after filters; dataframes will be empty for RQ analysis.")


➡ Fetching commits, pipelines & comments for getsentry/sentry PR #90516 (dataset id=2486573779)

➡ Fetching commits, pipelines & comments for getsentry/sentry PR #87963 (dataset id=2419106029)

➡ Fetching commits, pipelines & comments for oven-sh/bun PR #18585 (dataset id=2425248848)

➡ Fetching commits, pipelines & comments for oven-sh/bun PR #20612 (dataset id=2613893429)

➡ Fetching commits, pipelines & comments for oven-sh/bun PR #16857 (dataset id=2303501996)


Following Github server redirection from /repos/RockChinQ/LangBot to /repositories/575321313



➡ Fetching commits, pipelines & comments for RockChinQ/LangBot PR #1256 (dataset id=2427616889)

➡ Fetching commits, pipelines & comments for TracecatHQ/tracecat PR #1213 (dataset id=2609611207)

➡ Fetching commits, pipelines & comments for bruin-data/bruin PR #470 (dataset id=2356985296)

➡ Fetching commits, pipelines & comments for tokens-studio/figma-plugin PR #3402 (dataset id=2564432253)

➡ Fetching commits, pipelines & comments for liam-hq/liam PR #1994 (dataset id=2588963649)

➡ Fetching commits, pipelines & comments for vllm-project/vllm PR #16440 (dataset id=2452623588)

➡ Fetching commits, pipelines & comments for OpenHFT/Chronicle-Core PR #684 (dataset id=2260441374)

➡ Fetching commits, pipelines & comments for janbjorge/pgqueuer PR #390 (dataset id=2556468457)

➡ Fetching commits, pipelines & comments for seasonedcc/remix-forms PR #272 (dataset id=2269202548)

➡ Fetching commits, pipelines & comments for herbie-fp/herbie PR #1231 (dataset id=2537690761)

➡ Fetching commit

Following Github server redirection from /repos/microsoft/qsharp to /repositories/593773927



➡ Fetching commits, pipelines & comments for microsoft/qsharp PR #2530 (dataset id=2596620305)

➡ Fetching commits, pipelines & comments for stack-auth/stack-auth PR #546 (dataset id=2394225726)

➡ Fetching commits, pipelines & comments for crewAIInc/crewAI PR #2397 (dataset id=2400016065)

➡ Fetching commits, pipelines & comments for crewAIInc/crewAI PR #2136 (dataset id=2337334370)

➡ Fetching commits, pipelines & comments for crewAIInc/crewAI PR #2137 (dataset id=2337335339)

➡ Fetching commits, pipelines & comments for calcom/cal.com PR #19021 (dataset id=2308221415)

➡ Fetching commits, pipelines & comments for calcom/cal.com PR #20034 (dataset id=2389511160)

➡ Fetching commits, pipelines & comments for calcom/cal.com PR #20496 (dataset id=2432868443)

➡ Fetching commits, pipelines & comments for calcom/cal.com PR #20080 (dataset id=2392888093)

➡ Fetching commits, pipelines & comments for calcom/cal.com PR #21855 (dataset id=2597070258)

➡ Fetching commits, pipelines & comments

In [10]:
# Remove PRs where commits only touch markdown files (no code changes)
print("\nChecking for PRs with markdown-only changes...")

if len(df_details) == 0:
    print("  No commit detail rows; skipping markdown-only filter.")
    print(f"  Unique PRs retained (unchanged): {len(filtered_pr_ids):,}")
else:
    df_details["__is_markdown"] = df_details["filename"].str.lower().str.endswith((".md", ".markdown"))

    md_only_pr_ids = set(
        df_details.groupby("pr_id")["__is_markdown"]
        .agg(lambda s: bool(len(s)) and s.all())
        .pipe(lambda s: s[s].index)
    )

    print(f"  PRs with only markdown files: {len(md_only_pr_ids):,}")
    if md_only_pr_ids:
        df_details = df_details[~df_details["pr_id"].isin(md_only_pr_ids)].copy()
        df_commits = df_commits[~df_commits["pr_id"].isin(md_only_pr_ids)].copy()
        df_runs = df_runs[~df_runs["pr_id"].isin(md_only_pr_ids)].copy()
        df_issue_comments = df_issue_comments[~df_issue_comments["pr_id"].isin(md_only_pr_ids)].copy()
        df_review_comments = df_review_comments[~df_review_comments["pr_id"].isin(md_only_pr_ids)].copy()

    filtered_pr_ids = set(df_details["pr_id"].unique())
    df_details = df_details.drop(columns=["__is_markdown"])

    print(f"  Unique PRs retained after markdown-only filter: {len(filtered_pr_ids):,}")
    print("\nTotals after markdown-only filter:")
    print("  Commit rows (ai_pr_commits):", len(df_commits))
    print("  Detail rows (ai_pr_commit_details):", len(df_details))
    print("  Run rows (ai_pr_workflow_runs):", len(df_runs))
    print("  Issue comment rows (ai_pr_issue_comments):", len(df_issue_comments))
    print("  Review comment rows (ai_pr_review_comments):", len(df_review_comments))



Checking for PRs with markdown-only changes...
  PRs with only markdown files: 0
  Unique PRs retained after markdown-only filter: 83

Totals after markdown-only filter:
  Commit rows (ai_pr_commits): 210
  Detail rows (ai_pr_commit_details): 615
  Run rows (ai_pr_workflow_runs): 207
  Issue comment rows (ai_pr_issue_comments): 202
  Review comment rows (ai_pr_review_comments): 126


In [9]:
# ===================== SAVE TO PARQUET =====================
df_commits.to_parquet("./human_pr_commits.parquet", index=False)
df_details.to_parquet("./human_pr_commit_details.parquet", index=False)
df_runs.to_parquet("./human_pr_workflow_runs.parquet", index=False)
df_issue_comments.to_parquet("./human_pr_issue_comments.parquet", index=False)
df_review_comments.to_parquet("./human_pr_review_comments.parquet", index=False)

print("\nSaved: human_pr_commits.parquet")
print("Saved: human_pr_commit_details.parquet")
print("Saved: human_pr_workflow_runs.parquet")
print("Saved: human_pr_issue_comments.parquet")
print("Saved: human_pr_review_comments.parquet")
print(f"\nSkipped {skipped_count} PRs due to errors or missing data.")


Saved: human_pr_commits.parquet
Saved: human_pr_commit_details.parquet
Saved: human_pr_workflow_runs.parquet
Saved: human_pr_issue_comments.parquet
Saved: human_pr_review_comments.parquet

Skipped 0 PRs due to errors or missing data.


In [12]:
# Pr commit details for human PRs
human_pr_commit_details = pd.read_parquet("./human_pr_workflow_runs.parquet")
human_pr_commit_details.head()

,run_id,pr_id,pr_number,workflow_id,workflow_name,head_branch,head_sha,event,status,conclusion,created_at,updated_at,run_attempt,url
0,14714884717,2486573779,90516,3020796,sentry pull request bot,leander/rollout-io,91d120e06968062bca53859a962c3b41456606fa,pull_request,completed,skipped,2025-04-28 18:17:41+00:00,2025-04-28 18:17:46+00:00,1,https://github.com/getsentry/sentry/actions/runs/14714884717
1,14714884704,2486573779,90516,24420232,CodeQL,leander/rollout-io,91d120e06968062bca53859a962c3b41456606fa,pull_request,completed,success,2025-04-28 18:17:41+00:00,2025-04-28 18:28:12+00:00,1,https://github.com/getsentry/sentry/actions/runs/14714884704
2,14714884711,2486573779,90516,33200914,self-hosted,leander/rollout-io,91d120e06968062bca53859a962c3b41456606fa,pull_request,completed,success,2025-04-28 18:17:41+00:00,2025-04-28 18:23:16+00:00,1,https://github.com/getsentry/sentry/actions/runs/14714884711
3,14714884686,2486573779,90516,1585211,migrations,leander/rollout-io,91d120e06968062bca53859a962c3b41456606fa,pull_request,completed,success,2025-04-28 18:17:41+00:00,2025-04-28 18:18:06+00:00,1,https://github.com/getsentry/sentry/actions/runs/14714884686
4,14714884710,2486573779,90516,23572706,Enforce License Compliance,leander/rollout-io,91d120e06968062bca53859a962c3b41456606fa,pull_request,completed,success,2025-04-28 18:17:41+00:00,2025-04-28 18:19:01+00:00,1,https://github.com/getsentry/sentry/actions/runs/14714884710
